# Geometric Deep Learning Project - Towards Sparse Hierarchical Graph Classifiers

*Alessia Ruggeri*

### Implementation of the paper *Towards Sparse Hierarchical Graph Classifiers* tested on Enzymes, Proteins and D&D biological datasets using Tensorflow 2.0.

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import os,sys,inspect
import networkx as nx
import numpy as np
import scipy
import scipy.sparse as sp
import matplotlib.pyplot as plt
import time
import math

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Input, Dense, Flatten, Activation, Dropout, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model
from sklearn.utils import shuffle

from load_data import read_graphfile

np.random.seed(0)

In [0]:
### Unzip datasets folders

# !unzip -o data.zip
!unzip -o data_ENZYMES.zip
# !unzip -o data_PROTEINS.zip
# !unzip -o data_DD.zip
# !unzip -o data_COLLAB.zip

In [0]:
### Load datasets from data.zip

# print("\nLoading ENZYMES...")
# graphs_ENZYMES = read_graphfile(datadir="data", dataname="ENZYMES", max_nodes=None)

# print("\nLoading DD...")
# graphs_DD = read_graphfile(datadir="data", dataname="DD", max_nodes=None)

# print("\nLoading PROTEINS...")
# graphs_PROTEINS = read_graphfile(datadir="data", dataname="PROTEINS", max_nodes=None)

# print("\nLoading COLLAB...")
# graphs_COLLAB = read_graphfile(datadir="data", dataname="COLLAB", max_nodes=None)


### Load datasets from data_DATASET.zip

print("\nLoading ENZYMES...")
graphs_ENZYMES = read_graphfile(datadir="data_ENZYMES", dataname="ENZYMES", max_nodes=None)

# print("\nLoading PROTEINS...")
# graphs_PROTEINS = read_graphfile(datadir="data_PROTEINS", dataname="PROTEINS", max_nodes=None)

# print("\nLoading DD...")
# graphs_DD = read_graphfile(datadir="data_DD", dataname="DD", max_nodes=None)

# print("\nLoading COLLAB...")
# graphs_COLLAB = read_graphfile(datadir="data_COLLAB", dataname="COLLAB", max_nodes=None)

print("\nDone.")

In [0]:
### Generic functions

def get_numberof_features(dataset_name):
  if dataset_name == "ENZYMES":
    return 18
  elif dataset_name == "PROTEINS":
    return 1
  return None

def preprocess_features(features):
  '''Row-normalize feature matrix and convert it to dense representation'''
  rowsum = np.array(features.sum(1))
  r_inv = np.power(rowsum, -1).flatten()
  r_inv[np.isinf(r_inv)] = 0.
  r_mat_inv = sp.diags(r_inv)
  features = r_mat_inv.dot(features)
  return features

def get_node_features_matrix(graph):
  '''It returns the node feature matrix of the graph with already preprocessed features'''
  Xdict = nx.get_node_attributes(graph, 'feat')
  X = np.array([Xdict[i] for i in range(nx.number_of_nodes(graph))])
  X = preprocess_features(X)
  X = X.astype(np.float32)
  return X

def get_adjacency_matrix(graph):
  '''It returns the adjacency matrix of the graph with inserted self-loops'''
  A = nx.adjacency_matrix(graph)
  A = np.array(A + np.eye(A.shape[0]))
  A = sp.coo_matrix(A.astype(np.float32))
  return A

def get_normalization_matrix(A):
  '''It returns the normalized adjacency matrix of the graph'''
  degrees = np.array(np.sum(A.todense(), axis=1)).flatten()
  degrees = np.power(degrees, -1)
  degrees[np.isinf(degrees)] = 0
  degrees = degrees.astype(np.float32)
  D = sp.diags(degrees, offsets=0).tocoo()
  return D

def get_normalized_adjacency_matrix(A):
  D = get_normalization_matrix(A)
  A_norm = D @ A
  return A_norm.tocoo()

def get_graphs_labels(dataset):
  '''It returns the class labels of all the graphs in the dataset'''
  labels = []
  for graph in dataset:
    labels.append(graph.graph['label'])
  labels = np.array([[labels[i]] for i in range(len(labels))])
  return labels

def dot(x, y, sparse=False):
  '''Wrapper for tf.matmul (sparse vs dense)'''
  if sparse:
      res = tf.sparse.sparse_dense_matmul(x, y)
  else:
      res = tf.matmul(x, y)
  return res
  
def convert_sparse_matrix_to_sparse_tensor(coo):
  indices = np.transpose(np.array([coo.row, coo.col]))
  return tf.SparseTensor(indices, coo.data.astype(np.float32), coo.shape)

def convert_nparray_to_sparse_tensor(nparray):
  tf_tensor = tf.constant(nparray)
  idx = tf.where(tf.not_equal(tf_tensor, 0))
  sparse_tensor = tf.SparseTensor(idx, tf.gather_nd(tf_tensor, idx), tf_tensor.get_shape())
  return sparse_tensor

def convert_tf_tensor_to_sparse_tensor(tf_tensor):
  idx = tf.where(tf.not_equal(tf_tensor, 0))
  sparse_tensor = tf.SparseTensor(idx, tf.gather_nd(tf_tensor, idx), tf_tensor.get_shape())
  return sparse_tensor

def one_hot_encoding(data, n_classes):
    '''It one-hot encode data'''
    targets = np.array(data).reshape(-1)
    targets = np.eye(n_classes)[targets]
    return targets
  

In [0]:
### Execution functions

def convert_dataset_to_lists(dataset):
  feat = []
  adj = []
  for graph in dataset:
    X = get_node_features_matrix(graph)
    A = get_adjacency_matrix(graph)
    A_norm = get_normalized_adjacency_matrix(A)
    feat.append(X)
    adj.append(A_norm)
  return feat, adj

def create_batch_elements(X, A):
  '''It takes X and A lists and creates respective stack of nodes, block diagonal adjacency matrix and graph idx array'''
  X_stack = np.vstack(X)
  A_diag = sp.block_diag(A)
  A_diag = convert_sparse_matrix_to_sparse_tensor(A_diag)
  A_diag = tf.sparse.reorder(A_diag)
  n_nodes = np.array([a.shape[0] for a in A])
  graph_idx = np.repeat(np.arange(len(n_nodes)), n_nodes)
  return X_stack, A_diag, graph_idx

def batch_generator(data, batch_size=32):
  '''It takes a list of arrays or matrices and it yields batches of given size'''
  len_data = len(data[0])
  batches_per_epoch = math.ceil(len_data/batch_size)
  for batch in range(batches_per_epoch):
    start = batch * batch_size
    end = min(start+batch_size, len_data)
    out = [item[start:end] for item in data]
    yield out

# @tf.function
def train_step(data, labels):
  with tf.GradientTape() as tape:
    predictions = model(data)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
  
# @tf.function
def val_step(data, labels):
  predictions = model(data)
  t_loss = loss_object(labels, predictions)

  val_loss(t_loss)
  val_accuracy(labels, predictions)
  
# @tf.function
def test_step(data, labels):
  predictions = model(data)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
### Define the layers


class Convolutional(Layer):
  
  def __init__(self, F, F_1, regularizer, **kwargs):
    self.F = F
    self.F_1 = F_1
    self.regularizer = regularizer
    super(Convolutional, self).__init__(**kwargs)

  def build(self, input_shape):
    self.W1 = self.add_weight(name='W1', 
                             shape=(self.F, F_1),
                             initializer='he_normal',
                             regularizer=self.regularizer,
                             trainable=True)

    self.W2 = self.add_weight(name='W2', 
                             shape=(self.F, F_1),
                             initializer='he_normal',
                             regularizer=self.regularizer,
                             trainable=True)

    super(Convolutional, self).build(input_shape)  # Be sure to call this at the end
    
  def kernel(self, X, A):
    
    res = dot(A, X, sparse=True)
#     res = (N, F) -> AX
    res = dot(res, self.W1, sparse=False)
#     res = (N, F_1) -> AXW1
    skip_connection = dot(X, self.W2, sparse=False)
#     skip_connection = (N, F_1) -> XW2
    
    res = tf.math.add(res, skip_connection)
    res = ReLU()(res)
#     res = (N, F_1) -> sigma(AXW1 + XW2)
    
    return res
   
  def call(self, inputs):
    X = inputs[0]
    A = inputs[1]

    res = self.kernel(X, A)

    return res

  def compute_output_shape(self, input_shape):
    return (input_shape[0][0], self.F_1)

  
class GlobalAvgPooling(Layer):
  
  def __init__(self, **kwargs):
    super(GlobalAvgPooling, self).__init__(**kwargs)
    
  def build(self, input_shape):
    super(GlobalAvgPooling, self).build(input_shape)  # Be sure to call this at the end
  
  def call(self, inputs):
    nodes_feat = inputs[0]
    idx = inputs[1]
    
    res = tf.math.segment_mean(nodes_feat, idx)
    idx = tf.range(res.shape[0])
    
    return res, idx
  
  def compute_output_shape(self, input_shape):
    return (len(np.unique(idx)), self.F_1)
  
  
class GlobalSumPooling(Layer):
  
  def __init__(self, **kwargs):
    super(GlobalSumPooling, self).__init__(**kwargs)
    
  def build(self, input_shape):
    super(GlobalSumPooling, self).build(input_shape)  # Be sure to call this at the end
  
  def call(self, inputs):
    nodes_feat = inputs[0]
    idx = inputs[1]
    
    res = tf.math.segment_sum(nodes_feat, idx)
    
    return res
  
  def compute_output_shape(self, input_shape):
    return (len(np.unique(idx)), self.F_1)
  
  
class HierarchicalPooling(Layer):
  
  def __init__(self, F, pooling_ratio, regularizer, **kwargs):
    self.F = F
    self.k = pooling_ratio
    self.regularizer = regularizer
    super(HierarchicalPooling, self).__init__(**kwargs)
  
  def build(self, input_shape):
    
    self.p = self.add_weight(name='p', 
                             shape=(self.F, 1),
                             initializer='he_normal',
                             regularizer=self.regularizer,
                             trainable=True)
    
    super(HierarchicalPooling, self).build(input_shape)  # Be sure to call this at the end
  
  def call(self, inputs):
    X = inputs[0]
    A = inputs[1]
    idx = inputs[2]
    A = tf.sparse.to_dense(A)
    
#   Generate projection scores
    p_norm = tf.math.l2_normalize(self.p, axis=0)
    proj = tf.math.divide(self.p, p_norm)
    y = dot(X, proj, sparse=False)
#     y = (N, 1)
    
    feat = []
    adj = []
    indices = []
    for g in range(tf.size(tf.unique(idx)[0])):
#     Select only nodes of one graph
      gg = tf.reshape(tf.where(tf.equal(idx, g)), [-1])
      X_g = tf.gather(X, indices=gg)
      A_g = tf.gather(tf.gather(A, gg, axis=0),  gg, axis=1)
      idx_g = tf.gather(idx, gg, axis=0)
      y_g = tf.gather(y, gg, axis=0)
      
#     Generate indices of nodes to keep for that graph
      N = X_g.shape[0]
      to_keep = tf.cast(tf.math.ceil(N*self.k), dtype=tf.int32)
      i = tf.argsort(y_g, axis=0, direction='DESCENDING')
      i = tf.reshape(i, [-1])[:to_keep]
      
#     Filter X, A and idx of the graph using i to keep only nodes with higher projection score
      X_g = tf.gather(X_g, indices=i)
      A_g = tf.gather(tf.gather(A_g, i, axis=0),  i, axis=1)
      idx_g = tf.gather(idx_g, i, axis=0)
      
      y_tilde = tf.tanh(tf.gather(y_g, i, axis=0))
      X_tilde = tf.multiply(X_g, y_tilde)
      
      feat.append(X_tilde)
      adj.append(A_g)
      indices.append(idx_g)
    
#   Rebuild X, A and idx for the entire batch
    X_stack = tf.concat(feat, axis=0)
    A_diag = sp.block_diag(adj)
    A_diag = convert_sparse_matrix_to_sparse_tensor(A_diag)
    A_diag = tf.sparse.reorder(A_diag)
    idx_seq = tf.concat(indices, axis=-1)
    
    return X_stack, A_diag, idx_seq


class MyModel(Model):
  def __init__(self, F, F_1, pooling_ratio, n_classes, dropout, reg):
    super(MyModel, self).__init__()
    
    self.conv1 = Convolutional(F=F, F_1=F_1, regularizer=reg)
    self.avgpool1 = GlobalAvgPooling()
    self.hpool1 = HierarchicalPooling(F=F_1, pooling_ratio=pooling_ratio, regularizer=reg)
    self.conv2 = Convolutional(F=F_1, F_1=F_1, regularizer=reg)
    self.avgpool2 = GlobalAvgPooling()
    self.hpool2 = HierarchicalPooling(F=F_1, pooling_ratio=pooling_ratio, regularizer=reg)
    self.conv3 = Convolutional(F=F_1, F_1=F_1, regularizer=reg)
    self.avgpool3 = GlobalAvgPooling()
    self.hpool3 = HierarchicalPooling(F=F_1, pooling_ratio=pooling_ratio, regularizer=reg)
    self.sumpool = GlobalSumPooling()
    self.flat = Flatten()
    self.drop = Dropout(dropout)
    self.dense = Dense(n_classes, activation='softmax', kernel_regularizer=reg)

  def call(self, inputs):
    X = inputs[0]
    A = inputs[1]
    idx = inputs[2]
    
#   First conv-pool block
    X_conv1 = self.conv1([X, A])
    X_hpool1, A_hpool1, idx_hpool1 = self.hpool1([X_conv1, A, idx])
    avg1, idx_avgpool1 = self.avgpool1([X_hpool1, idx_hpool1])
    s1 = tf.concat([avg1, X_hpool1], axis=0)
    idx1 = tf.concat([idx_avgpool1, tf.cast(idx_hpool1, dtype=tf.int32)], axis=-1)
#   Second conv-pool block
    X_conv2 = self.conv2([X_hpool1, A_hpool1])
    X_hpool2, A_hpool2, idx_hpool2 = self.hpool2([X_conv2, A_hpool1, idx_hpool1])
    avg2, idx_avgpool2 = self.avgpool2([X_hpool2, idx_hpool2])
    s2 = tf.concat([avg2, X_hpool2], axis=0)
    idx2 = tf.concat([idx_avgpool2, tf.cast(idx_hpool2, dtype=tf.int32)], axis=-1)
#   Third conv-pool block
    X_conv3 = self.conv3([X_hpool2, A_hpool2])
    X_hpool3, A_hpool3, idx_hpool3 = self.hpool3([X_conv3, A_hpool2, idx_hpool2])
    avg3, idx_avgpool3 = self.avgpool3([X_hpool3, idx_hpool3])
    s3 = tf.concat([avg3, X_hpool3], axis=0)
    idx3 = tf.concat([idx_avgpool3, tf.cast(idx_hpool3, dtype=tf.int32)], axis=-1)
#   Readout block
    s = tf.concat([s1, s2, s3], axis=0)
    idx = tf.concat([idx1, idx2, idx3], axis=-1)
    res = self.sumpool([s, idx])
#   MLP for classification
    res = self.drop(res)
    res = self.dense(res)
    
    return res

In [8]:
dataset = graphs_ENZYMES
dataset_name = "ENZYMES"
# dataset = graphs_PROTEINS
# dataset_name = "PROTEINS"

labels = get_graphs_labels(dataset)
n_classes=len(np.unique(labels))

dataset, labels = shuffle(dataset, labels)

train_n = len(dataset)//100 * 80
val_n = len(dataset)//100 * 15

(x_val, y_val) = dataset[0:val_n], labels[0:val_n]
(x_train, y_train) = dataset[val_n:train_n], labels[val_n:train_n]
(x_test, y_test) = dataset[train_n:], labels[train_n:]

print('train:', len(x_train))
print('val:', len(x_val))
print('test:', len(x_test))

X_train, A_train = convert_dataset_to_lists(x_train)
X_val, A_val = convert_dataset_to_lists(x_val)
X_test, A_test = convert_dataset_to_lists(x_test)

y_train = one_hot_encoding(y_train, n_classes)
y_val = one_hot_encoding(y_val, n_classes)
y_test = one_hot_encoding(y_test, n_classes)


train: 390
val: 90
test: 120


In [0]:
# Hyperparameters

epochs=1000
batch_size = 64

dropout = 0.3
learning_rate = 5e-4
reg = l2(5e-3)

F = get_numberof_features(dataset_name)
F_1 = 128
pooling_ratio = 0.8

In [0]:
# # layer = HierarchicalPooling(F=F, pooling_ratio=pooling_ratio)
# # layer = GlobalAvgPooling()
# model = MyModel(F, F_1, pooling_ratio, n_classes, dropout, reg)
# generator = batch_generator([X_val, A_val, y_val], batch_size=45)

# for X, A, y in generator:
#   X_batch, A_batch, idx_batch = create_batch_elements(X, A)
  
# #   X, idx = layer([X_batch, idx_batch])
# #   print(idx)
# #   X, A, idx = layer([X_batch, A_batch, idx_batch])
#   result = model([X_batch, A_batch, idx_batch])
  
#   print(result)

In [0]:
### Set up model and training variables
model = MyModel(F, F_1, pooling_ratio, n_classes, dropout, reg)

loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


In [12]:
### Training loop
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')

start_time = time.time()
epoch_time = start_time

for epoch in range(epochs):
  if epoch != 0 and epoch % 100 == 0:
    end_time = time.time()
    tmp = end_time - epoch_time
    print(f"Time of execution of the last 100 epochs: around {round(tmp/60)} min")
    epoch_time = end_time
  
  X_train, A_train, y_train = shuffle(X_train, A_train, y_train)
  train_generator = batch_generator([X_train, A_train, y_train], batch_size=batch_size)
  val_generator = batch_generator([X_val, A_val, y_val], batch_size=batch_size)
  
  for X, A, y in train_generator:
    X_batch, A_batch, idx_batch = create_batch_elements(X, A)
    train_step([X_batch, A_batch, idx_batch], y)

  for X, A, y in val_generator:
    X_batch, A_batch, idx_batch = create_batch_elements(X, A)
    val_step([X_batch, A_batch, idx_batch], y)

  template = 'Epoch {} \t train_loss: {:.4f}\t train_accuracy: {:.4f}\t val_loss: {:.4f}\t val_accuracy: {:.4f}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         val_loss.result(),
                         val_accuracy.result()*100))

  
end_time = time.time()
tmp = end_time - start_time
print(f"Time of execution of all the epochs: around {round(tmp/60)} min")

W0511 14:02:59.218892 140185121892224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_grad.py:425: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


Epoch 1 	 train_loss: 6.8156	 train_accuracy: 14.6154	 val_loss: 5.0911	 val_accuracy: 12.2222
Epoch 2 	 train_loss: 5.6810	 train_accuracy: 17.6923	 val_loss: 4.9105	 val_accuracy: 10.5556
Epoch 3 	 train_loss: 5.0973	 train_accuracy: 18.3761	 val_loss: 4.6272	 val_accuracy: 10.7407
Epoch 4 	 train_loss: 4.7834	 train_accuracy: 19.1026	 val_loss: 4.3888	 val_accuracy: 11.3889
Epoch 5 	 train_loss: 4.5564	 train_accuracy: 19.6923	 val_loss: 4.2714	 val_accuracy: 10.6667
Epoch 6 	 train_loss: 4.3799	 train_accuracy: 19.8291	 val_loss: 4.0942	 val_accuracy: 10.9259
Epoch 7 	 train_loss: 4.1615	 train_accuracy: 20.4029	 val_loss: 3.9798	 val_accuracy: 11.1111
Epoch 8 	 train_loss: 3.9768	 train_accuracy: 20.8974	 val_loss: 3.8651	 val_accuracy: 11.6667
Epoch 9 	 train_loss: 3.9059	 train_accuracy: 21.1681	 val_loss: 3.7518	 val_accuracy: 12.4691
Epoch 10 	 train_loss: 3.7799	 train_accuracy: 20.7692	 val_loss: 3.6604	 val_accuracy: 13.2222
Epoch 11 	 train_loss: 3.7152	 train_accuracy: 20

KeyboardInterrupt: ignored

In [13]:
### Test loop
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')

test_generator = batch_generator([X_test, A_test, y_test], batch_size=batch_size)
val_generator = batch_generator([X_val, A_val, y_val], batch_size=batch_size)

for X, A, y in test_generator:
  X_batch, A_batch, idx_batch = create_batch_elements(X, A)
  test_step([X_batch, A_batch, idx_batch], y)
  
for X, A, y in val_generator:
  X_batch, A_batch, idx_batch = create_batch_elements(X, A)
  val_step([X_batch, A_batch, idx_batch], y)

template = 'test_loss: {:.4f}\t test_accuracy: {:.4f}\t val_loss: {:.4f}\t val_accuracy: {:.4f}'
print (template.format(test_loss.result(),
                       test_accuracy.result()*100,
                       val_loss.result(),
                       val_accuracy.result()*100))

test_loss: 1.7750	 test_accuracy: 47.5000	 val_loss: 1.7255	 val_accuracy: 42.2222
